In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math
import os
import hmm1
import sklearn
import pickle
import time

# Read in files

In [56]:
# Load emission probability table
ems_prob = pd.read_csv('ems_prob.csv')

In [57]:
# Read in data (from pickle file)
file = open('drug_df_filtered','rb')
drug_df = pickle.load(file)

# Get subset for training

In [58]:
december_sales = drug_df[drug_df['date'] > datetime(2018,12,30)]
split_idx = 100
train = december_sales.sort_values(by = 'date').iloc[:split_idx, :]
test = december_sales.sort_values(by = 'date').iloc[split_idx:, :]

In [59]:
#Training set
train_hidden = list(train['vendor_name'])
train_observed = list(train['drug_prediction'])

#Testing set
test_hidden = list(test['vendor_name'])
test_observed = list(test['drug_prediction'])

# Format Data

In [79]:
# Get unique drugs and vendors
_, unique_vendors = pd.factorize(train_hidden, sort = True)
_, unique_drugs = pd.factorize(train_observed, sort = True)

# Function checks if item is in the set
def isInSet(item, myset):
    return(item in myset)

# Get subset of emission probability rows
vendor_in_set = ems_prob['Vendor Name'].apply(isInSet, myset = unique_vendors)
ems_prob_subset = ems_prob[vendor_in_set]

# Get subset of emission probability cols
ems_prob_subset = ems_prob_subset[unique_drugs]

In [87]:
vendor_dict = dict(zip(list(range(0,len(unique_vendors))),unique_vendors))
vendor_dict

{0: '-Lenas-BioLaden-',
 1: '0a74ad41873d53e50e9dce228c96433c',
 2: '550d05ab240ec337038af814ff0de287',
 3: 'Alagbada726',
 4: 'AlanCuring420_UK',
 5: 'BlackSheepSquadron',
 6: 'CaptainPirate',
 7: 'ChillChem',
 8: 'CryptoICE',
 9: 'DCdutchconnectionUK',
 10: 'DMT-lovestore',
 11: 'DRUGPOOL',
 12: 'ETHER_MART',
 13: 'GRUENEWALDSCAVE',
 14: 'GreenSupreme',
 15: 'GrimReefer420',
 16: 'HeinekenExpress',
 17: 'HonestTrader',
 18: 'Kman3000',
 19: 'MoonBanana',
 20: 'Multi',
 21: 'NordicBars',
 22: 'Perfect.Shrooms',
 23: 'PopeyeUK',
 24: 'Recettear',
 25: 'ScarlettsK',
 26: 'Shiny.Flakes',
 27: 'Smokey1884',
 28: 'WA2018',
 29: 'YOURDEALER',
 30: 'bangalow',
 31: 'blueviking',
 32: 'fun-gee',
 33: 'g00d00',
 34: 'sthompson',
 35: 'tinyrock'}

In [61]:
# Create data frame to be used by Baum Welch (one-hot encoding)
train_hidden_encoded = pd.get_dummies(train_hidden)
train_hidden_encoded = np.matrix(train_hidden_encoded)

train_observed_array, _ = pd.factorize(train_observed)
train_observed_array = train_observed_array.reshape(-1,1)

In [77]:
labs, u = pd.factorize(train_hidden)
u

array(['WA2018', 'BlackSheepSquadron', 'Perfect.Shrooms',
       '-Lenas-BioLaden-', 'sthompson', 'YOURDEALER', 'tinyrock',
       'GrimReefer420', 'HonestTrader', 'ChillChem', 'Alagbada726',
       'Kman3000', 'CryptoICE', 'DCdutchconnectionUK', 'PopeyeUK',
       'MoonBanana', 'NordicBars', '0a74ad41873d53e50e9dce228c96433c',
       'HeinekenExpress', 'DMT-lovestore', 'Recettear', 'Shiny.Flakes',
       '550d05ab240ec337038af814ff0de287', 'Smokey1884', 'blueviking',
       'g00d00', 'Multi', 'ETHER_MART', 'bangalow', 'GRUENEWALDSCAVE',
       'AlanCuring420_UK', 'CaptainPirate', 'GreenSupreme', 'ScarlettsK',
       'DRUGPOOL', 'fun-gee'], dtype=object)

In [62]:
new_hmm = hmm1.Hmm(states = unique_vendors, observations = unique_drugs, ems_prob = ems_prob_subset)

# Apply Baum-Welch algorithm

In [67]:
# Parameters for Baum-Welch
n_iter = 10
n_train = 5

# Run Baum-Welch algorithm
start = time.time()
start_prob, trans_prob = new_hmm.hmm_BaumWelch(data = train_hidden_encoded, n_iter = n_iter, n_train = n_train)
stop = time.time()
elapsed = stop-start
print(elapsed)

12.690099000930786


In [68]:
print(trans_prob)

[[0.04574929 0.02593045 0.03482071 ... 0.02195449 0.02276143 0.021077  ]
 [0.04582816 0.02592235 0.03485163 ... 0.02192894 0.02273941 0.02104759]
 [0.04578163 0.02592713 0.03483339 ... 0.02194401 0.0227524  0.02106494]
 ...
 [0.04586124 0.02591895 0.03486459 ... 0.02191822 0.02273018 0.02103526]
 [0.04585358 0.02591973 0.03486159 ... 0.0219207  0.02273232 0.02103811]
 [0.04587023 0.02591802 0.03486811 ... 0.02191531 0.02272767 0.0210319 ]]


# Apply Viterbi algorithm

In [70]:
vendor_preds = new_hmm.hmm_Viterbi(obs_seq = train_observed_array, start_prob = start_prob, trans_prob = trans_prob)

The best logprob is:  -355.8788277256775


/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:412: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T
/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:412: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T
/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:412: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T
/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:412: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T
/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:412: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T
/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:412: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, n

In [71]:
vendor_preds

array([ 4,  4,  5, 32, 32,  4,  4,  4,  4, 11, 11, 11, 11, 11, 11, 11, 26,
       14, 14, 14,  4, 11, 26, 26,  4,  4,  4,  4,  4,  4, 10,  6,  6,  6,
        6, 25,  4,  4, 11,  9,  9,  9, 26, 25, 25,  6,  4, 26, 26, 26,  4,
        4,  4,  4,  4, 14,  4, 14,  4,  4, 14, 14, 14, 14,  4,  4,  4,  9,
       10, 10,  9, 26, 26, 26,  4, 31,  6, 26, 30, 30, 12,  4, 30, 30,  9,
       31, 31,  4,  6,  6,  4, 25, 11, 11, 11, 11, 11, 32, 26, 26])